# Проект 1.
# Исследование надёжности заёмщиков.

<a name="1"></a>
# Содержание/план

1  [Описание проекта и постановка задачи.](#1)

2  [Импорт библиотек, загрузка данных, изучение данных.](#2)

3  [Предобработка данных](#3)

* 3.1  [Импорт библиотек, загрузка данных, изучение данных.](#31)
* 3.2  [Изучение и подготовка данных.](#32)
  * 3.2.1  [data_arc — данные об электродах](#321)
  * 3.2.2  [data_bulk — данные о подаче сыпучих материалов (объём)](#322)
  * 3.2.3  [data_bulk_time — данные о подаче сыпучих материалов (время)](#323)
  * 3.2.4  [data_gas — данные о продувке сплава газом](#324)
  * 3.2.5  [data_temp — результаты измерения температуры](#325)
  * 3.2.6  [data_wire — данные о проволочных материалах (объём)](#326)
  * 3.2.7  [data_wire_time — данные о проволочных материалах (время)](#327)
* 3.3  [Удаление данных с ошибками.](#33)
* 3.4  [Создание дополнительных признаков.](#34)
  * 3.4.1  [Температура](#341)
  * 3.4.2  [Нагревание](#342)
  * 3.4.3  [Добавление сыпучих и проволочных материалов](#343)

4  [Исследовательский анализ данных](#4)

* 4.1  [Температура.](#41)
  * 4.1.1  [Начальные данные](#411)
  * 4.1.2  [Измененные данные](#412)
* 4.2  [Нагревание.](#42)
* 4.3  [Добавление сыпучих и проволочных материалов, продувка.](#43)

5  [Подготовка к обучению.](#5)

* 5.1  [Объединение таблиц.](#51)
* 5.2  [Выявление корреляции.](#52)

6  [Обучение и проверка моделей.](#6)

* 6.1  [Подготовка к обучению.](#61)
* 6.2  [LinearRegression.](#62)
* 6.3  [DecisionTreeRegressor.](#63)
* 6.4  [RandomForestRegressor.](#64)
* 6.5  [LightGBM.](#65)
* 6.6  [CatBoost.](#66)
* 6.7  [Сравнение моделей.](#67)
    
7  [Общий вывод.](#7)

<a name="1"></a>
## Описание проекта и постановка задачи.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных:
*    children — количество детей в семье
*    days_employed — общий трудовой стаж в днях
*    dob_years — возраст клиента в годах
*    education — уровень образования клиента
*    education_id — идентификатор уровня образования
*    family_status — семейное положение
*    family_status_id — идентификатор семейного положения
*    gender — пол клиента
*    income_type — тип занятости
*    debt — имел ли задолженность по возврату кредитов
*    total_income — ежемесячный доход
*    purpose — цель получения кредита

<a name="2"></a>
## Импорт библиотек, загрузка данных, изучение данных. 

In [ ]:
import pandas as pd
from pymystem3 import Mystem

In [ ]:
#функция для приведения стажа к единому виду
def days_years_employed (days_employed): #возвращает значения в годах стажа
    '''
    принимает значение стажа в 
    '''
    
        if days_employed>36500:          #если значение больше 36500 дней то считаем что оно указано в часах
            return (days_employed)/(365*24)
        return abs(days_employed/(365))

In [2]:
data=pd.read_csv('data.csv')

In [3]:
data.info()

#сколько отрицательных значений в days_employed? ->  15906
print()
print('количество отрицательных значений days_employed:', len(data[data['days_employed']<0]))




data['years_employed']=data['days_employed'].apply(days_years_employed) #добавляю столбец со стажем в годах

#какие значения в 'children'?
print()
print('какие значения в "children"')
print(data['children'].value_counts())

data['children']=abs(data['children']) #меняю значение 'children' с -1 на 1
data['children']=data['children'].replace(20, 2) #меняю значение 'children' с 20 на 2

#Какие значения в 'gender'?
print()
print('какие значения в "gender"')
print(data['gender'].value_counts())
data=data[data['gender']!='XNA'] #удаляю строку с гендером XNA

#сколько нулевых значений возраста? ->    101
print()
print('количество нулевых значений возраста:', data[data['dob_years'] == 0]['dob_years'].count())

data=data[data['dob_years']!=0] #удаляю строки с 0 возрастом т.к. их меньше 0,5%

#какие значения в 'income_type'? -> 4 малочисленные группы: предприниматель, безработный, студент, в декрете.
print()
print('какие значения в "income_type"')
print(data['income_type'].value_counts())

data=data[data['income_type']!='предприниматель']#удаляю малочисленные группы 'income_type'
data=data[data['income_type']!='безработный']    #удаляю малочисленные группы 'income_type'
data=data[data['income_type']!='студент']        #удаляю малочисленные группы 'income_type'
data=data[data['income_type']!='в декрете']      #удаляю малочисленные группы 'income_type'

#какая доля строк с NaN? ->   10%
print()
nan_days_employed=data[data['days_employed'].isna()==True] #пропуски в days_employed совпадают с пропусками в total_income
nan_total_income=data[data['total_income'].isna()==True] #пропуски в total_income совпадают с пропусками в days_employed
print('NaN в {:.0%} строк'.format(len(nan_days_employed)/len(data))) 

#не связаны ли пропуски с типом занятости? возможно стаж и зарплата не указана для определенной группы -> нет
#print(data['income_type'].value_counts()) #подсчет всех типов занятости
#print(nan_total_income['income_type'].value_counts()) # 4 разных типов занятости для пропусков по доходу в месяц
#print(nan_days_employed['dob_years'].value_counts()) # много разных возрастов для пропусков по трудовому стажу

data.drop(['days_employed'], axis='columns', inplace=True) #удаляю столбец 'days_employed'
data.drop(['education_id'], axis='columns', inplace=True) #удаляю столбец 'education_id'
data.drop(['family_status_id'], axis='columns', inplace=True) #удаляю столбец 'family_status_id'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

количество отрицательных значений days_employed: 15906

какие значения в "children"
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4      

<div class="alert alert-block alert-danger">
<h2>Комментарий от ревьювера (недочет, который нужно доработать)</h2> 
    
Выводы есть, а отображения на экране нет)
    
1. Стоит выводить на экран, все что ты исследуешь, заказчику тоже захочется посмотреть) Скажем так, какие ваши доказательства?
    
2. Наши проекты очень приближены к реальным. У аналитиков есть тактя поговорка, что работа аналитика заканчивается только тогда, когда начальник говорит "хватит") Завтра тебе дадут задание исследовать зависимость пенсионеров, что будешь делать) 
    
3. Если ты не используешь какой либо код, не стоит его комментировать - просто удали, чтоб он не "шумел")
    
4. Используй метод info(), для отображения общей информации о таблице, это очень удобно)
    
5. Для замен отрицательных на положительные есть  https://pythonz.net/references/named/abs/    


<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ ОТ УЧЕНИКА: </b>
Доработал.
</div>

### Вывод

Необходима предобработка. Названия столбцов однотипные, для удобной работы замены не требуют. Входные данные "сырые": разнородные('education'), есть пропуски('days_employed', 'total_income'), нулевые значения ('dob_years') и некорректно заполненные ('children', 'days_employed'), часть значений в столбце 'days_employed' (для пенсионеров) вероятно имеет значения трудовой стаж в днях*24, а остальные данные имеют отрицательные значения. Трудовой стаж удобнее анализировать в годах а не в днях. Данные о трудовом стаже собраны в новый столбец 'years_employed' (трудовой стаж в годах), столбец 'days_employed' удалил. В столбце 'children' есть значения "-1" (либо клиенты "должны" кому-то ребенка (например Prawo niespodzianki как у Сапковского) либо ошибка знака) - заменил на 1 и значения "20" (вероятно ошибка порядка) - заменил на 2. Удалил 4 малочисленные группы из 'income_type'.

<div class="alert alert-block alert-success">
<h2>Комментарий от ревьювера V2</h2> 

Отлично)

### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
#блок обрабатывает пропуски в 'total_income' и 'years_employed'
nan_colomn='total_income'      #переменная задает столбец с пропуском
group_colomn='income_type'     #переменная задает столбец по которому будем группировать для поиска медианы
y=data.groupby(group_colomn)[nan_colomn].median() #series со значениями медиан по группе

def median_groupby(row):       #функция получает строку
    try:
        x=row[group_colomn]    #выдергиваем значение столбца группировки в строке
        z=y[x]                 #получаю значение медианы из series для нужной группы
        return z               #возвращает значение медианы для нужной группы
    except:
        return 'error z'

#Заполняю 'total_income' медианой соответствующей возрастной группы
data[nan_colomn]=data[nan_colomn].fillna(data.apply(median_groupby, axis=1))  

#Заполняю 'years_employed' медианой соответствующего типа занятости
nan_colomn='years_employed'    #переменная задает столбец с пропуском
group_colomn='dob_years'       #переменная задает столбец по которому будем группировать для поиска медианы
y=data.groupby(group_colomn)[nan_colomn].median() #series со значениями медиан по группе
data[nan_colomn]=data[nan_colomn].fillna(data.apply(median_groupby, axis=1))



### Вывод

Пропуски в столбцах 'days_employed' (трудовой стаж) и 'total_income' (доход в месяц) составляют 10% и носят случайный характер. Значения трудового стажа заполнил медианой соответствующей возрастной группы, значения дохода - медианой соответствующего типа занятости (заполнение медианой соответствующего уровня образования на конечные результаты влияет незначительно).

<div class="alert alert-block alert-success">
<h2>Комментарий от ревьювера (все здОрово)</h2> 
    
Отличная замена. Использование функций - это очень хорошая практика, пригодится в реальной работе. Тип зароботка - пожалуй, самая правильная категория, молодец)
</div>

### Замена типа данных

In [5]:
#меняю столбец 'years_employed' и 'total_income' на целочисленный методом astype т.к. он более универсальный чем to_numeric
data['years_employed']=data['years_employed'].astype('int64') 
data['total_income']=data['total_income'].astype('int64')

### Вывод

Стаж и доход перевел в целочисленные значения.

## Обработка дубликатов

In [6]:
#привожу данные в столбце 'education' к нижнему регистру
data['education']=data['education'].str.lower()

data=data.drop_duplicates().reset_index(drop=True) #удаляю дубликаты

### Вывод

Данные в столбце 'education' записаны разнородно (с использованием ВЕРХНЕГО нижнего и Смешанного регистра). Привел к нижнему регистру. Удалил дубликаты. Частично дубликаты связаны с наличием одинаковых строк с различными вариантами написания в столбце 'education', природа остальных не ясна. 

<div class="alert alert-block alert-warning">
<h2>Комментарий от ревьювера (принимаем к размышлению)</h2> 

В принципе, верно) Обрати внимание, что у нас в таблице нет уникальный айдишников, либо других идентификационных параметров. Такие дубликаты могут быть случаным совпадением)
</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ ОТ УЧЕНИКА: </b>
Согласен. Буду учитывать.
</div>

### Лемматизация

In [ ]:
#лемматизирую, "склеиваю", отбрасываю \n
m=Mystem()

data['purpose_lemm']=data['purpose'].apply(m.lemmatize).str.join('').str.strip()

print(data['purpose_lemm'].value_counts()) #4 цели: автомобиль, свадьба, недвижимость / жилье, образование

### Вывод

Данные содержат всего 4 цели получения кредита: автомобиль, свадьба, недвижимость / жилье, образование

<div class="alert alert-block alert-success">
<h2>Комментарий от ревьювера (все здОрово)</h2> 
    
Молодец, очень элегантное решение. Самая сложная часть решена на отлично, выше всяких похвал)
</div>

### Категоризация данных

In [ ]:
#Функция возвращает категорию цели кредита
def purpose_groups (row):
    if 'автомобиль' in row:
        return 'автомобиль'
    if 'свадьба' in row:
        return 'свадьба'
    if 'недвижимость' in row:
        return 'недвижимость'
    if 'жилье' in row:
        return 'недвижимость'
    if 'образование' in row:
        return 'образование'
    return 'УТОЧНИТЬ'

data['purpose_group']=data['purpose_lemm'].apply(purpose_groups)

data.drop(['purpose'], axis='columns', inplace=True) #удаляю столбец 'purpose'
data.drop(['purpose_lemm'], axis='columns', inplace=True) #удаляю столбец 'purpose_lemm'

#определяю 25, 50, 75 процентили 'total_income'
#quantile25=data['total_income'].quantile(0.25) # -> 107629.0
#quantile50=data['total_income'].quantile(0.50) # -> 142594.0
#quantile75=data['total_income'].quantile(0.75) # -> 195826.25

#Функция возвращает категорию дохода
def total_income_groups (x):
    try:
        if x>200000:
            return 'более 200 тыс./мес.'
        if x>150000:
            return '150-200 тыс./мес.'
        if x>100000:
            return '100-150 тыс./мес.'
        if x<=100000:
            return 'менее 100 тыс./мес.'
    except:
        return 'ERROR'

#добавляем столбец с уровнем дохода
data['income_group']=data['total_income'].apply(total_income_groups)

### Вывод

Добавил столбец 'purpose_group' - категория цели кредита. Разбил заемщиков на 4 категории по уровню дохода, добавил категорию в столбец 'income_group'.

<div class="alert alert-block alert-success">
<h2>Комментарий от ревьювера (все здОрово)</h2> 
    
Класс- категоризация - супер)
</div>

<div class="alert alert-block alert-warning">
<h2>Комментарий от ревьювера (принимаем к размышлению)</h2> 

Замечательная часть - все верно, расчеты проведены на высоком уровне, только старайся значения не хардкорить. Ты прописываешь цифры напрямую, и если поменять датасет, код придется переписывать. Но если использовать квантили в самой функции - то это автоматизирует процесс, можно будет не переписывать код при замене данных)
</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ ОТ УЧЕНИКА: </b>
Изначально разделял на 4 группы по квантилям, но в этом случае надо вводить "относительные" категории дахода (низкий, ниже среднего, выше среднего, высокий) и при оценке дохода потенциального заемщика относить его к одной из этих категорий.
    Выбрал категоризацию в абсолютных значениях а не в относительных.
</div>

<div class="alert alert-block alert-success">
<h2>Комментарий от ревьювера V2</h2> 

Точно)

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
#средний % должников
debtall=data['debt'].sum()/data['debt'].count()
print('% должников по всему датасету: {:.2%}'.format(debtall))

#% должников среди бездетных
debtchildfree=data[data['children']==0]['debt'].sum()/data[data['children']==0]['debt'].count()
print('% должников среди бездетных: {:.2%}'.format(debtchildfree))

#средний % должников среди заемщиков с детьми
debtchild=data[data['children']!=0]['debt'].sum()/data[data['children']!=0]['debt'].count()
print('% должников среди заемщиков с детьми: {:.2%}'.format(debtchild))
print()

#% должников в зависимости от количества детей
print('% должников в зависимости от количества детей:')
childrendebt=data.groupby('children')['debt'].agg(['sum', 'count'])
childrendebt['debt%']=childrendebt['sum']/childrendebt['count']*100
print(childrendebt)


### Вывод

Наличие детей (вне зависимости от их количества) увеличивает вероятность возникновения задолженности по кредиту. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
#% должников в зависимости от семейного положения
print('% должников в зависимости от семейного положения:')
familydebt=data.groupby('family_status')['debt'].agg(['sum', 'count'])
familydebt['debt%']=familydebt['sum']/familydebt['count']*100
print(familydebt.sort_values(by='debt%'))

### Вывод

Наиболее добросовестные заемщики - бывшие ранее в браке и находящиеся сейчас в официальном браке. Риск невозврата кредита для холостых, а также заемщиков с неоформленными супружескими отношениями возрастает.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
#% должников в зависимости от уровня дохода
print('% должников в зависимости от уровня дохода:')
incomedebt=data.groupby('income_group')['debt'].agg(['sum', 'count'])
incomedebt['debt%']=incomedebt['sum']/incomedebt['count']*100
print(incomedebt.sort_values(by='debt%'))

### Вывод

Наиболее дисциплинированы заемщики с доходом свыше 200 тыс./мес., а также категория с доходом менее 100 тыс./мес (возможно связано с меньшим размером кредита). Риск невозврата для средних категорий возрастает.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
#% должников в зависимости от цели кредита
print('% должников в зависимости от цели кредита:')
purposedebt=data.groupby('purpose_group')['debt'].agg(['sum', 'count'])
purposedebt['debt%']=purposedebt['sum']/purposedebt['count']*100
print(purposedebt.sort_values(by='debt%'))

### Вывод

Наименьший риск в категории кредитов на недвижимость, наибольший у автокредитов и кредитов на образование.

<div class="alert alert-block alert-danger">
<h2>Комментарий от ревьювера (недочет, который нужно доработать)</h2> 
    
Супер, ты прекрасно освоил группировки и сложные расчеты) Хочу обратить внимание на очень лаконичные выводы, замечательные рассуждения, грамотный код)
    
Я обратил внимание, что в своем исследовании ты не используешь сводные таблицы. А это очень важный и популярный инструмент, без которого в реальной работе просто никуда) Давай исследуем зависимости при помощи сводных таблиц. Пускай в индексах у нас будет категория, а в столбцах - число заемщиков в данной категории, число должников и процент должников. Не посчитай это придиркой, но мн важно видеть, что ты освоил сводные таблицы,дальше без них ну никак)
</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ ОТ УЧЕНИКА: </b>
Спасибо. Теоретическая часть не дала полного представления о возможностях сводных таблиц - поэтому не было понимания как их использовать в исследовании. Применил ниже.
</div>

In [ ]:
data_pivot=data.pivot_table(index = ['family_status'], columns= 'gender', values= 'debt', aggfunc=['count', 'sum', 'mean'])
print(data_pivot)
print()
data_pivot2=data.pivot_table(index = ['income_type'], values= 'debt', aggfunc=['count', 'sum', 'mean'])
print(data_pivot2)

<div class="alert alert-block alert-success">
<h2>Комментарий от ревьювера V2</h2> 

Шикарно)

### Вывод

Риски по группам с различным семейным положением значительно отличаются в зависимости от пола заемщика. Это необходимо учитывать.

### Шаг 4. Общий вывод

Исследование показало что статистика о платежеспособности клиентов может быть использована для построения модели кредитного скоринга. 
_____
Информация о заемщике влияет на риски в следующем по значимости порядке: 

* ##### семейное положение
Наиболее добросовестные заемщики - бывшие ранее в браке и находящиеся сейчас в официальном браке. Риск невозврата кредита для холостых, а также заемщиков с неоформленными супружескими отношениями возрастает.
* ##### наличие детей
Наличие детей (вне зависимости от их количества) увеличивает вероятность возникновения задолженности по кредиту.
* ##### цель получения кредита
Наименьший риск в категории кредитов на недвижимость, наибольший у автокредитов и кредитов на образование.
* ##### уровень дохода 
Наиболее дисциплинированы заемщики с доходом свыше 200 тыс./мес., а также категория с доходом менее 100 тыс./мес (возможно связано с меньшим размером кредита). Риск невозврата для средних категорий возрастает.

Также необходимо принимать во внимание прочую информацию о клиенте (к примеру статистика по группам значительно отличается для мужчин и для женщин).

<div class="alert alert-block alert-warning">
<h2>Комментарий от ревьювера (принимаем к размышлению)</h2> 

Старайся делать более развернутые выводы, заказчики очень большой упор делают на финальный вывод, и хотят видеть в ней ну прям всю информацию)
</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ ОТ УЧЕНИКА: </b>
поправил, спасибо.
</div>

## <span style="color:magenta">В заключении)</span>

Мне очень понравилась твоя работа! Все рассуждения верные. Очень логичные и осмысленные. Ничего лишнего, все по полкам. 

Ты показываешь хорошее владение всем изучаемым в модуле материалом. Уверенно пользуешься pandas, строишь отличные наглядные таблички. Делаешь четкие обоснованные выводы.

Поисправляй недочеты (оформление, предобработка и сводные таблицы), и проект будет просто класс!

Короче, молодец! Жду твоих исправлений)

<img src="https://avatars.mds.yandex.net/get-pdb/2402172/12f53009-3c0e-4655-87c6-606d139bbf8e/s1200?webp=false" width="300">

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.